<a href="https://colab.research.google.com/github/PeLaNo2/Petchsaritpong/blob/main/Gemini_classify_xls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import userdata
import os
os.environ["GEMINI_API_KEY"]=userdata.get('gemini_api')

In [7]:
import csv
import os
from google import genai
from google.genai import types
import time
import re
from tqdm import tqdm


def generate_items(company, PRODUCT_CODE, product_category, existing_items):
    """
    Generates item numbers (4-5 ideally) using Gemini, with Google Search.
    Prioritizes existing items.
    """
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.0-flash"

    item_list = """
    PRODUCT / CATEGORY / ITEM 	CATEGORY NO	ITEM NO
Others /　Grocery	147	1388
Housewares /　Hardware, DIY, House Improvements and Maintenance /　Hardware & DIY Products	115	1192
Housewares /　Household Products /　Cleaning Equipment	116	1649
Housewares /　Kitchen & Dining /　Bakeware & Cookware	117	1156
Housewares /　Kitchen & Dining /　Kitchenware	117	1202
Housewares /　Kitchen & Dining /　Tableware	117	1238
Housewares /　Kitchen & Dining /　Kitchen Appliances, Tools & Gadgets	117	1652
Home Décor /　Garden, Outdoor, Lawn, Patio and Terrace /　Garden & Outdoor Accessories	114	1189
Home Décor /　Ornaments and Interior Accessories /　Home Décor	119	1301
Home Décor /　Soft Furnishings and Home Textiles /　Home Textile	125	1302
Genearl Home Textiles /　Bedroom Textiles /　Bed Linens	157	1395
Genearl Home Textiles /　Bedroom Textiles /　Bedspreads	157	1396
Genearl Home Textiles /　Bedroom Textiles /　Decorative Pillows	157	1397
Genearl Home Textiles /　Bedroom Textiles /　Beddings	157	1399
Genearl Home Textiles /　Mats	158, 160	1404
Genearl Home Textiles /　Curtain Tiebacks	158	1406
Genearl Home Textiles /　Bathroom Accessories (Textiles)	160	1413
Genearl Home Textiles /　Kitchen & Bathroom Textiles /　Kitchen & Dining Textiles	160	1634
Genearl Home Textiles /　Fabrics (Home Textiles) /　Mixed Fabrics - Garment	161	1427
Pet Care Products	122	1221
Pet Toys	122	1276
Gifts and Premiums /　Aromatic Products /　Aromatic Products	102	1053
Gifts and Premiums /　Arts and Crafts & Decorative Items /　Arts & Crafts	103	1055
Gifts and Premiums /　Arts and Crafts & Decorative Items /　Handmade Decoratives	103	2045
Gifts and Premiums /　Fashion Accessories and Apparel /　Leather Goods	110	1098
Gifts and Premiums /　Giftwrap & Packaging /　Packaging Tools, Machinery and Equipment	120	1629
Gifts and Premiums /　Premiums /　Giveaways	123	1085
Gifts and Premiums /　Premiums /　Souvenirs	123	1132
Baby and Nursery Items /　Nursery Items	105	1294
Baby and Nursery Items /　Babywear & Footwear	105	1648
Baby and Nursery Items / Baby Toys /　Baby Toys	130	1254
Baby and Nursery Items + Children Toys  /  Children's Care Products	105, 131	1297
Toys & Games /    Die Cast/ Mechanical Toys & Action Figures + Hobbies & Collectibles   /  Action Figures	132, 314	1253
Toys & Games /　Educational Toys & Games /　Puzzles, Board & Card Games	133	1280
Toys & Games /　General Merchandise / Multiple Products  /　Licensed Products	135	1272
Toys & Games /　Outdoor & Sporting Items /　Playground Equipments	136	1278
Toys & Games /　Toy Parts & Accessories /　Toy Parts & Accessories	138	1289
Toys & Games /　Soft Toys & Dolls /　Dolls	140	1264
Toys & Games /　Other Toys /　Other Toys	998	1305
General Stationery /　Calendars	126, 151	1063
General Stationery /　Folders, Files & Filing Accessories	126, 148	1080
General Stationery /　Greeting Cards	126, 151	1089
General Stationery /   Name card Storage	126, 149	1109
General Stationery /　Paper Products	126	1114
General Stationery /　 Writing Instruments	126, 148	1149
General Stationery /　Office Supplies /　Office Accessories	149	1325
Jewellery /　Fine Jewellery /　Diamond Jewellery	162	1384
Jewellery /　Fine Jewellery /　Gemset Jewelry	162	1820
Jewellery /　Fine Jewellery /　Gold Jewelry	162	1821
Jewellery /　Fine Jewellery /　Jade Jewelry	162	1822
Jewellery /　Fine Jewellery /　Pearl Jewelry	162	1823
Jewellery /　Fine Jewellery /　Platinum Jewelry	162	1824
Jewellery /　Fine Jewellery /　Silver Jewellery	162	1825
Jewellery /　Fine Jewellery /　Stainless Steel Jewelry	162	1826
Jewellery /　Fine Jewellery /　Antique Jewelry	162	1835
Jewellery /　Fine Jewellery /　Finished Jewellery	162	1839
Jewellery /　Fine Jewellery /　Jewellery Accessories	162	1841
Jewellery /　Gold & Silver /　Pure Gold Jewellery	166	1354
Jewellery /　Gold & Silver /　Silver Jewellery	166	1382
Jewellery /　Jewellery Display & Packaging /　Display	232	1834
Jewellery /　Jewellery Display & Packaging /　Jewellery Tools & Instrument	232	1843
General Garment /　Eyewear	145	1873
General Garment /　Children's Apparel, Bags and Footwear	155	1308
General Garment /　Garment /　Menswear	155	1356
General Garment /　Womenswear	155	1357
General Garment /　Babies and Children's Wear	155	1358
General Garment /　Casual wear	155	1359
General Garment /　Knit Wear	155	1799
General Garment /　Wedding Dress	155	2182
General Garment /　Garment /　Jacket	155	2194
General Garment /　Garment /　Sweater	155	2195
General Garment /　Garment /　T-shirt	155	2196
General Garment /　Garment /  SHIRTS	155	2236
General Garment /　Garment /  CUSTOM CLOTHING	155	2237
General Garment /　Garment /  STREETWEAR	155	2238
General Garment /　Garment + Sportswear and Sports Shoes  /  SPORT BRA & LEGGINGS	155, 306	2239
General Garment /　Garment + Sportswear and Sports Shoes /  YOGA WEAR	155, 306	2240
General Garment /　Garment /  TROUSERS	155	2241
General Garment /　Garment /  SHORTS	155	2242
General Garment /　Garment /  HOODIES	155	2243
General Building Materials /　Building Materials /　Metal building materials	176	1477
General Building Materials /　Building Materials /　Chemical Industry Building Materials	176	1478
General Building Materials /　Building Materials /　Glass building materials	176	1479
General Building Materials /　Building Materials /　Cement products	176	1480
General Building Materials /　Building Materials /　Fireproof material	176	1481
General Food /　Food /　MEAT	185	1544
General Food /　Food /　egg+ dairy	185	1545
General Food /　Food /　SEAFOOD	185	1546
General Food /　Food /　FRUITS	185	1547
General Food /　Food /　VEGAN	185	1548
General Food /　Food /　Condiment	185	1549
General Food /　Food /　Food Additives	185	1552
General Food /　Food /　Bee Products	185	1553
General Food /　Drinks /　Beer	186	1554
General Food /　Drinks /　Spirits	186	1555
General Food /　Drinks /　Wines	186	1556
General Food /　Drinks /　Coffee	186	1557
General Food /　Drinks /　Fruit Juice	186	1558
General Food /　Drinks /　Drinking Water	186	1559
General Food /　Drinks /　Alcoholic	186	1952
General Food /　Drinks /　Non-alcoholic Beverages	186	1955
General Food /　Tea /　Food Packaging 	207	1662
General Food /　Oils	290	1543
General Food /　Grain Products	290	1953
General Food /　Dairy Products	290	1956
General Food /　Chilled & Frozen Food /　Chilled Seafood	291	1954
General Food /       Canned Food	292	1814
General Food /    Snacks 	293, 295	1551
General Food /　Sweets & Confectionery /　Candies and Confectionery Products	295	1550
General Food /　Honey	296	2184
General Machinery /　Machine Accessories 	268
General Machinery /　Factory Machines	271
General Machinery /　Factory Machines /　Industrial Automation & Intelligent Manufacturing	271	2232
General Machinery /　Factory Machines /　Processing Machinery Equipment	271	2233
General Machinery /　Construction Used Machines 	275
General Machinery /　Food Machinery	339
Lighting /　Office Workplace Lighting	208	1665
Lighting /　Commercial Lighting /  Hotel, Restaurant & Shop Lighting	208	1690
Lighting /　Home Lighting  /　Table / Desk Lamps	209	1668
Lighting /　Home Lighting /   Floor Lamps	209	1669
Lighting /　Home Lighting /  Night Lamps	209	1670
Lighting /　Lighting /  Ceiling Lamps & Lighting	209	1691
Lighting /　Decorative Lighting /  Chandeliers	210	1673
Lighting /　Decorative Lighting /  Decorative Table Lamps	210	1674
Lighting /　Lighting /  Fibre Optic Lights	210	1782
Lighting /　Professional & Outdoor Lighting /  Stage Lighting	211	1675
Lighting /　Professional & Outdoor Lighting /  Special Venue Lighting	211	1676
Lighting /　Professional & Outdoor Lighting /  Outdoor Lighting Equipment	211	1677
Lighting /　Professional & Outdoor Lighting /  Garden & Landscape Lighting	211	1678
Lighting /　LED & Energy-saving Lighting /  LED Light Bulbs	212	1679
Lighting /　LED & Energy-saving Lighting /  LED Lighting	212	1680
Lighting /　LED & Energy-saving Lighting /  Energy-saving Light Bulbs	212	1681
Lighting /　LED & Energy-saving Lighting /  Energy-saving Lighting System	212	1682
Lighting /　Solar CELL	212	1729
Lighting /  LIGHTING SOLUTION	344
 Lighting /    Commercial Lighting + Home Lighting  /  Lighting For Emergency & Safety	208, 209	1786
Lighting /  Commercial Lighting + Professional & Outdoor Lighting  /  Spotlight	208, 211	1788
Lighting /   Home Lighting + Professional & Outdoor Lighting  /  Wall Lamps	209, 211	1667
Consumer Electronics /　Audio-visual Products /  LCD Display	214	1693
Consumer Electronics /　Audio-visual Products /  Electronic Games & Entertainment	214	1699
Consumer Electronics /　Audio-visual Products /  Digital Audio Systems	214	1731
Consumer Electronics /　Personal Electronics /  Electronic Smart Devices & Accessories	216	1701
Servicing Industries /　Exhibition Agent /  Exhibition & Event Planning	220	1838
Servicing Industries /　Exhibition Organizer	221
Servicing Industries /　Consultants Company  /  Development Company	312	2042
Design Studio / Designer Services	242
Footwear /　Fashion Footwear /  Mens Shoes	248	1863
Footwear /　Fashion Footwear /  Ｗomens Ｓhoes	248	1864
Footwear /　Fashion Footwear /  Childrens Shoes	248	1865
Footwear /　Casual Footwear /  Slipper and Flip-flops	250	1769
Footwear /　Casual Footwear /  Sandals	250	1871
Footwear /　Professional Footwear /  Work and Safety Footwear	251	1868
IT /　IT Solutions  /  Website design	269	1926
 IT /     IT Solutions  /  Interactive / digital kiosks	269	1930
IT /　IT Solutions  /  Logo design	269	1931
IT /　IT Solutions  /  Educational learning APP	269	2049
IT /　IT Solutions  /  Learning online	269	2050
IT /　Software /  system /  Mobile application	270	1927
IT /　Software /  system /  360 photography / video	270	1928
IT /　Software / system /  VR (virtual reality)	270	1929
Festive, Christmas and Seasonal /  Balloons		1058
Festive, Christmas and Seasonal / Christmas Decorations		1069
Travel and Luggage /　Travel Accessories /  Backpacks	315	1056
Cosmetics & Personal Care /　Cosmetic /  Skincare	297	1965
Cosmetics & Personal Care /　Personal Care /  Health Care Products	298	1964
Cosmetics & Personal Care /　Personal Care /  Body Care Products	298	1967
Halal  / Halal Beauty 	349
Halal  / Halal Fashion 	350
Halal  / Halal Wellness & Health 	351
Halal  / Halal Tourism 	352
Halal  / Halal  Food  / Halal Food 	353	1957
HANDICRAFT
Home Appliances  +   Kitchen Appliances /  Blenders	109, 201	1165
Home Appliances  +   Kitchen Appliances /  Coffee Makers	109, 201	1175
Home Appliances  +   Kitchen Appliances /  Rice Cookers	109, 201	1178
Home Appliances   +  Kitchen Appliances  /  Microwave Ovens 	109, 201	1214
Home Appliances    +    Kitchen Appliances / Refrigerators	109, 201	1226
Home Appliances    +    Kitchen Appliances /  Stoves & Ovens	109, 201	1235
"""

    Text_prompt = f"""
Company: {company}
PRODUCT CODE: {PRODUCT_CODE}
product_category: {product_category}

Based on the provided information and the list below, classify this company's products
according to the following ITEM NOs. Output *only* the ITEM NOs, separated by commas.
Choose up to 5 item numbers. Output ONLY numbers. Use Google Search to get
more context about the company and its products if needed.

{item_list}

Reason step by step, then provide the best matching ITEM NUMBER(S) within a <final_answer> tag.
For example: <final_answer>1192, 1202, 1649, 1652</final_answer> or <final_answer>1301, 1302, 1404, 1413</final_answer>.
"""

    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=Text_prompt),
            ],
        ),
    ]

    tools = [
        types.Tool(function_declarations=[types.FunctionDeclaration(
            name="google_search",
            description="Perform a Google Search for a given query.",
            parameters=types.Schema(
                type=types.Type.OBJECT,
                properties={
                    "query": types.Schema(type=types.Type.STRING, description="The search query.")
                },
                required=["query"],
            )
        )])
    ]

    generate_content_config = types.GenerateContentConfig(
        temperature=0.2,
        top_p=0.95,
        top_k=40,
        max_output_tokens=8192,
        tools=tools,
        response_mime_type="text/plain",
    )

    try:
        response = client.models.generate_content(
            model=model,
            contents=contents,
            config=generate_content_config,
        )

        match = re.search(r"<final_answer>(.*?)</final_answer>", response.text)
        if match:
            final_answer = match.group(1).strip()
            new_item_numbers = []
            for num_str in final_answer.split(','):
                num_str = num_str.strip()
                if num_str.isdigit():
                    new_item_numbers.append(num_str)

            return new_item_numbers

        else:
            print(f"Warning: <final_answer> tag not found.")
            return []

    except Exception as e:
        print(f"Error during Gemini inference: {e}")
        return []


# Correct Indentation:
def process_csv(input_file, output_file, start_row=1, end_row=None):
    """Processes the CSV, prioritizing existing items, adding new (up to 5)."""
    try: # Good practice to wrap file operations in try/except
        with open(input_file, 'r', encoding='cp1252') as infile, \
                open(output_file, 'w', newline='', encoding='utf-8') as outfile:

            # ---> START: All these lines MUST be indented under 'with' <---
            reader = csv.DictReader(infile)

            # Prepare fieldnames for the writer
            fieldnames = reader.fieldnames
            if fieldnames is None:
                 print(f"ERROR: Could not read headers from {input_file}. Is it empty or not a valid CSV?")
                 return # Exit if no headers

            # Check if 'ITEM' column exists, if not, add it to the list for the writer
            if 'ITEM' not in fieldnames:
                fieldnames = fieldnames + ['ITEM'] # Create a new list including 'ITEM'

            writer = csv.DictWriter(outfile, fieldnames=fieldnames)
            writer.writeheader() # Write the potentially modified headers

            # Wrap the reader with tqdm *here*, and use enumerate on tqdm(reader)
            # Note: total=end_row might be inaccurate if end_row is beyond file length
            # Consider removing 'total' or doing a first pass to count lines for accuracy
            print(f"Processing rows from {start_row}...") # Add some logging
            for row_count, row in tqdm(enumerate(reader, start=1), desc="Processing CSV"):
                if row_count < start_row:
                    # Need to write skipped rows too, ensuring all columns are present
                    output_row = {field: row.get(field, '') for field in fieldnames}
                    if 'ITEM' not in row: output_row['ITEM'] = '' # Ensure ITEM exists for writer
                    writer.writerow(output_row)
                    continue
                if end_row is not None and row_count > end_row:
                    break # Stop processing after end_row

                # --- Your row processing logic would go here ---
                # Example placeholder:
                company = row.get('COMPANY ', '').strip()
                PRODUCT_CODE = row.get('PRODUCT CODE', '').strip()
                product_category = row.get('PRODUCT CATEGORY', '').strip()
                existing_items_str = row.get('ITEM', '').strip() # Get existing ITEM if column was present
                existing_items = [item.strip() for item in existing_items_str.split(',') if item.strip().isdigit()]

                if not company or not PRODUCT_CODE or not product_category:
                     print(f"\nSkipping row {row_count} due to missing data.")
                     output_row = {field: row.get(field, '') for field in fieldnames}
                     if 'ITEM' not in row: output_row['ITEM'] = existing_items_str # Write original ITEM if present
                     elif not output_row.get('ITEM'): output_row['ITEM'] = '' # Ensure ITEM exists
                     writer.writerow(output_row)
                     continue

                time.sleep(4.1) # Consider moving delay right before API call if possible

                new_items = generate_items(company, PRODUCT_CODE, product_category, existing_items) # Call your Gemini function
                combined_items = existing_items.copy()
                for item in new_items:
                    if item.isdigit() and item not in combined_items and len(combined_items) < 5:
                        combined_items.append(item)

                final_items = [item for item in combined_items[:5] if item.isdigit()]

                # Prepare the row for writing, ensuring all fields exist
                output_row = {field: row.get(field, '') for field in fieldnames}
                output_row['ITEM'] = ", ".join(final_items) # Add the processed items

                writer.writerow(output_row)
                # ---> END: Indentation under 'with' ends here <---

    except FileNotFoundError:
        print(f"ERROR: Input file not found: {input_file}")
    except Exception as e:
        print(f"An error occurred during CSV processing: {e}")

# --- Make sure the rest of your script (generate_items, __main__) is
        # correctly defined ---

        # Wrap the reader with tqdm *here*, and use enumerate on tqdm(reader)
        for row_count, row in tqdm(enumerate(reader, start=1), desc="Processing CSV", total=end_row):
            if row_count < start_row:
                writer.writerow(row)
                continue
            if end_row is not None and row_count > end_row:
                break

            company = row.get('COMPANY', '')
            PRODUCT_CODE = row.get('PRODUCT CODE', '')
            product_category = row.get('PRODUCT CATEGORY', '')
            existing_items_str = row.get('ITEM', '')
            existing_items = [item.strip() for item in existing_items_str.split(',') if item.strip()]

            if not company or not PRODUCT_CODE or not product_category:
                print(f"Skipping row {row_count} due to missing data.")
                writer.writerow(row)
                continue

            new_items = generate_items(company, PRODUCT_CODE, product_category, existing_items)
            combined_items = existing_items.copy()

            for item in new_items:
                if item not in combined_items and len(combined_items) < 5:
                    combined_items.append(item)

            final_items = combined_items[:5]
            row['ITEM'] = ", ".join(final_items)

            writer.writerow(row)
            time.sleep(4.2)

if __name__ == "__main__":
    input_csv_file = "/content/Book(update by moke).csv"  # Your input CSV file
    output_csv_file = "output.csv"
    start_row = 121
    end_row = 125  # Or None to process to the end
    process_csv(input_file=input_csv_file, output_file=output_csv_file, start_row=start_row, end_row=end_row)
    print("Processing complete. Results saved to:", output_csv_file)

Processing rows from 121...


Processing CSV: 125it [00:00, 13417.14it/s]


Skipping row 121 due to missing data.

Skipping row 122 due to missing data.

Skipping row 123 due to missing data.

Skipping row 124 due to missing data.

Skipping row 125 due to missing data.
Processing complete. Results saved to: output.csv
